In [1]:
# Import necessary libraries
import pandas as pd
from prophet import Prophet
from IPython.display import clear_output
import os
import logging

c:\BATCOMPUTER'S Drive D\Python Lab\pyenv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Set logging level to WARNING to remove 'INFO' warnings from Prophet
logging.getLogger('prophet').setLevel(logging.WARNING)

In [3]:
# Read the data
data_import = pd.read_csv("NIFTY50_HD.csv", parse_dates=["Date"])
data_import

,Date,Open,High,Low,Close,Adj Close,Volume
0,2007-09-17,4518.450195,4549.049805,4482.850098,4494.649902,4494.649902,0.0
1,2007-09-18,4494.100098,4551.799805,4481.549805,4546.200195,4546.200195,0.0
2,2007-09-19,4550.250000,4739.000000,4550.250000,4732.350098,4732.350098,0.0
3,2007-09-20,4734.850098,4760.850098,4721.149902,4747.549805,4747.549805,0.0
4,2007-09-21,4752.950195,4855.700195,4733.700195,4837.549805,4837.549805,0.0
...,...,...,...,...,...,...,...
4164,2024-08-01,25030.949219,25078.300781,24956.400391,25010.900391,25010.900391,431300.0
4165,2024-08-02,24789.000000,24851.900391,24686.849609,24717.699219,24717.699219,345000.0
4166,2024-08-05,24302.849609,24350.050781,23893.699219,24055.599609,24055.599609,487000.0
4167,2024-08-06,24189.849609,24382.599609,23960.400391,23992.550781,23992.550781,312300.0


In [4]:
# Add a column of the day names
day_mapper = {0: "Monday", 1:"Tuesday", 2:"Wednesday", 3:"Thursday", 4:"Friday", 5:"Saturday", 6:"Sunday"}
data_import["DayOfWeek"] = data_import["Date"].map(lambda x: day_mapper[x.dayofweek])
data_import


,Date,Open,High,Low,Close,Adj Close,Volume,DayOfWeek
0,2007-09-17,4518.450195,4549.049805,4482.850098,4494.649902,4494.649902,0.0,Monday
1,2007-09-18,4494.100098,4551.799805,4481.549805,4546.200195,4546.200195,0.0,Tuesday
2,2007-09-19,4550.250000,4739.000000,4550.250000,4732.350098,4732.350098,0.0,Wednesday
3,2007-09-20,4734.850098,4760.850098,4721.149902,4747.549805,4747.549805,0.0,Thursday
4,2007-09-21,4752.950195,4855.700195,4733.700195,4837.549805,4837.549805,0.0,Friday
...,...,...,...,...,...,...,...,...
4164,2024-08-01,25030.949219,25078.300781,24956.400391,25010.900391,25010.900391,431300.0,Thursday
4165,2024-08-02,24789.000000,24851.900391,24686.849609,24717.699219,24717.699219,345000.0,Friday
4166,2024-08-05,24302.849609,24350.050781,23893.699219,24055.599609,24055.599609,487000.0,Monday
4167,2024-08-06,24189.849609,24382.599609,23960.400391,23992.550781,23992.550781,312300.0,Tuesday


# Method 1: Average Dip per Day

In [5]:
# Calculate the difference from the previous day
data_import["diff_from_previous_day"] = (data_import["Open"].diff() / data_import["Open"]) * 100
data_import

,Date,Open,High,Low,Close,Adj Close,Volume,DayOfWeek,diff_from_previous_day
0,2007-09-17,4518.450195,4549.049805,4482.850098,4494.649902,4494.649902,0.0,Monday,NaN
1,2007-09-18,4494.100098,4551.799805,4481.549805,4546.200195,4546.200195,0.0,Tuesday,-0.541824
2,2007-09-19,4550.250000,4739.000000,4550.250000,4732.350098,4732.350098,0.0,Wednesday,1.233996
3,2007-09-20,4734.850098,4760.850098,4721.149902,4747.549805,4747.549805,0.0,Thursday,3.898753
4,2007-09-21,4752.950195,4855.700195,4733.700195,4837.549805,4837.549805,0.0,Friday,0.380818
...,...,...,...,...,...,...,...,...,...
4164,2024-08-01,25030.949219,25078.300781,24956.400391,25010.900391,25010.900391,431300.0,Thursday,0.576287
4165,2024-08-02,24789.000000,24851.900391,24686.849609,24717.699219,24717.699219,345000.0,Friday,-0.976035
4166,2024-08-05,24302.849609,24350.050781,23893.699219,24055.599609,24055.599609,487000.0,Monday,-2.000384
4167,2024-08-06,24189.849609,24382.599609,23960.400391,23992.550781,23992.550781,312300.0,Tuesday,-0.467138


In [6]:
# Compare the different days to see which one has the biggest dip from the previous day
data_import.groupby("DayOfWeek")["diff_from_previous_day"].mean()

DayOfWeek
Friday      -0.064421
Monday       0.049775
Saturday          NaN
Sunday            NaN
Thursday     0.071950
Tuesday      0.035471
Wednesday    0.040872
Name: diff_from_previous_day, dtype: float64

# Method 2: Backtester Method

Tracks the number of shares that I own based on the number of shares that I bought when and what happened to the price on any given day

In [7]:
# Create a table of every day between the minimum date and maximum date to deal with holidays
dates = pd.date_range(start=data_import["Date"].min(), end=data_import["Date"].max())

In [8]:
# Add the name of the day of the week
date_table = pd.DataFrame(data={"Calendar Date": dates})
date_table["Weekday"] = date_table["Calendar Date"].map(lambda x: day_mapper[x.dayofweek])
date_table

,Calendar Date,Weekday
0,2007-09-17,Monday
1,2007-09-18,Tuesday
2,2007-09-19,Wednesday
3,2007-09-20,Thursday
4,2007-09-21,Friday
...,...,...
6165,2024-08-03,Saturday
6166,2024-08-04,Sunday
6167,2024-08-05,Monday
6168,2024-08-06,Tuesday


In [9]:
# Create a table of all dates and remove the weekends
full_calendar = pd.merge(left=date_table, right=data_import, how='left', left_on='Calendar Date', right_on='Date')
full_calendar = full_calendar[~full_calendar["Weekday"].isin(["Saturday", "Sunday"])]
full_calendar.rename(columns={"Date": "Trading day"}, inplace=True)
full_calendar

,Calendar Date,Weekday,Trading day,Open,High,Low,Close,Adj Close,Volume,DayOfWeek,diff_from_previous_day
0,2007-09-17,Monday,2007-09-17,4518.450195,4549.049805,4482.850098,4494.649902,4494.649902,0.0,Monday,NaN
1,2007-09-18,Tuesday,2007-09-18,4494.100098,4551.799805,4481.549805,4546.200195,4546.200195,0.0,Tuesday,-0.541824
2,2007-09-19,Wednesday,2007-09-19,4550.250000,4739.000000,4550.250000,4732.350098,4732.350098,0.0,Wednesday,1.233996
3,2007-09-20,Thursday,2007-09-20,4734.850098,4760.850098,4721.149902,4747.549805,4747.549805,0.0,Thursday,3.898753
4,2007-09-21,Friday,2007-09-21,4752.950195,4855.700195,4733.700195,4837.549805,4837.549805,0.0,Friday,0.380818
...,...,...,...,...,...,...,...,...,...,...,...
6163,2024-08-01,Thursday,2024-08-01,25030.949219,25078.300781,24956.400391,25010.900391,25010.900391,431300.0,Thursday,0.576287
6164,2024-08-02,Friday,2024-08-02,24789.000000,24851.900391,24686.849609,24717.699219,24717.699219,345000.0,Friday,-0.976035
6167,2024-08-05,Monday,2024-08-05,24302.849609,24350.050781,23893.699219,24055.599609,24055.599609,487000.0,Monday,-2.000384
6168,2024-08-06,Tuesday,2024-08-06,24189.849609,24382.599609,23960.400391,23992.550781,23992.550781,312300.0,Tuesday,-0.467138


In [10]:
# In order to account for holidays, take the next day's opening value as that's the price I'd buy the stock at
full_calendar = full_calendar.bfill(axis='rows').reset_index(drop=True)
full_calendar


,Calendar Date,Weekday,Trading day,Open,High,Low,Close,Adj Close,Volume,DayOfWeek,diff_from_previous_day
0,2007-09-17,Monday,2007-09-17,4518.450195,4549.049805,4482.850098,4494.649902,4494.649902,0.0,Monday,-0.541824
1,2007-09-18,Tuesday,2007-09-18,4494.100098,4551.799805,4481.549805,4546.200195,4546.200195,0.0,Tuesday,-0.541824
2,2007-09-19,Wednesday,2007-09-19,4550.250000,4739.000000,4550.250000,4732.350098,4732.350098,0.0,Wednesday,1.233996
3,2007-09-20,Thursday,2007-09-20,4734.850098,4760.850098,4721.149902,4747.549805,4747.549805,0.0,Thursday,3.898753
4,2007-09-21,Friday,2007-09-21,4752.950195,4855.700195,4733.700195,4837.549805,4837.549805,0.0,Friday,0.380818
...,...,...,...,...,...,...,...,...,...,...,...
4403,2024-08-01,Thursday,2024-08-01,25030.949219,25078.300781,24956.400391,25010.900391,25010.900391,431300.0,Thursday,0.576287
4404,2024-08-02,Friday,2024-08-02,24789.000000,24851.900391,24686.849609,24717.699219,24717.699219,345000.0,Friday,-0.976035
4405,2024-08-05,Monday,2024-08-05,24302.849609,24350.050781,23893.699219,24055.599609,24055.599609,487000.0,Monday,-2.000384
4406,2024-08-06,Tuesday,2024-08-06,24189.849609,24382.599609,23960.400391,23992.550781,23992.550781,312300.0,Tuesday,-0.467138


In [11]:
# There's a different number of each Weekday across 12 years
full_calendar["Weekday"].value_counts()

Weekday
Monday       882
Tuesday      882
Wednesday    882
Thursday     881
Friday       881
Name: count, dtype: int64

In [12]:
# Define a function to backtest as if we invested from the beginning
def day_backtester(day, amount_to_invest, data):
    temp_data = data[data["Weekday"] == day]
    temp_data["Shares Owned"] = amount_to_invest / temp_data["Open"]
    final_price = temp_data["Open"].iloc[-1]
    final_amount = temp_data["Shares Owned"].sum() * final_price
    formatted_final_amount = "${:,.2f}".format(final_amount)
    return formatted_final_amount

In [13]:
# Print the backtest results for each weekday
for i in full_calendar["Weekday"].unique():
    print(i, day_backtester(i, 750, full_calendar))

Monday $2,134,708.30
Tuesday $2,123,609.85
Wednesday $2,132,270.59
Thursday $2,194,472.33
Friday $2,174,292.71


C:\Users\harsh\AppData\Local\Temp\ipykernel_14976\417788892.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp_data["Shares Owned"] = amount_to_invest / temp_data["Open"]
C:\Users\harsh\AppData\Local\Temp\ipykernel_14976\417788892.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp_data["Shares Owned"] = amount_to_invest / temp_data["Open"]
C:\Users\harsh\AppData\Local\Temp\ipykernel_14976\417788892.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.


# Method 3: Facebook Prophet

In [14]:
# Define a function to predict using Prophet
def prophet_predictor(dataset):
    # Needs to take 3 weeks of data (Monday - Friday) and predict the 4th week
    m = Prophet()
    m.fit(dataset)
    future = m.make_future_dataframe(periods=7)
    forecast = m.predict(future)
    return forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].tail()

In [15]:
# Create an empty list to store forecast data
forecast_data = []

In [16]:
# Start the forecasting process
for i in full_calendar[full_calendar["Weekday"] == "Monday"]["Calendar Date"].iloc[402:].index:
    days_to_include = i - 15
    temp = full_calendar.iloc[days_to_include:i, [0, 3]]
    temp.rename(columns={"Calendar Date": "ds", "Open": "y"}, inplace=True)
    prophet_predictor(temp).to_pickle(f"three_week_forecast_files/{i}.pkl")
    clear_output(wait=True)
    print(i, "/", full_calendar[full_calendar["Weekday"] == "Monday"]["Calendar Date"].iloc[17:].index.max())


4405 / 4405


In [17]:
# Import and append all of the Pickle files
files_to_append = []
files_to_read = ["three_week_forecast_files/" + i for i in os.listdir("three_week_forecast_files")]
for i in files_to_read:
    files_to_append.append(pd.read_pickle(i))

In [18]:
# Combine all of the saved Pickle files
predicted_values = pd.concat(files_to_append).sort_values("ds")
predicted_values.rename(columns={"ds": "Predicted_Date", "yhat": "Prediction"}, inplace=True)
predicted_values

,Predicted_Date,Prediction,yhat_lower,yhat_upper
17,2015-06-01,8575.021365,8213.947965,8941.648179
18,2015-06-02,8761.231220,8179.342439,9330.138251
19,2015-06-03,8686.928961,7856.253036,9509.744302
20,2015-06-04,8729.862547,7631.586904,9834.665147
21,2015-06-05,8711.322155,7283.001654,10120.389524
...,...,...,...,...
17,2024-08-05,25073.360768,24905.420210,25237.521876
18,2024-08-06,25115.504724,24912.305277,25312.332240
19,2024-08-07,25092.616825,24810.929260,25357.580664
20,2024-08-08,25096.057724,24735.009250,25437.373756


In [19]:
# The lowest value day per week: Predicted Part 1
full_calendar["Week Monday"] = full_calendar["Calendar Date"] - pd.to_timedelta(full_calendar["Calendar Date"].dt.dayofweek, unit='d')
predicted_values["Week Monday"] = predicted_values["Predicted_Date"] - pd.to_timedelta(predicted_values["Predicted_Date"].dt.dayofweek, unit='d')
min_predicted_values = predicted_values.groupby("Week Monday")["Prediction"].min().reset_index()
min_predicted_values

,Week Monday,Prediction
0,2015-06-01,8575.021365
1,2015-06-08,7465.349171
2,2015-06-15,8020.570306
3,2015-06-22,8332.151146
4,2015-06-29,8490.187404
...,...,...
475,2024-07-08,24166.213674
476,2024-07-15,24675.658313
477,2024-07-22,24940.122499
478,2024-07-29,23915.048014


In [20]:
# The lowest value day per week: Predicted Part 2
predicted_values_output = pd.merge(min_predicted_values, predicted_values, how="left", on=["Week Monday", "Prediction"])
predicted_values_output = predicted_values_output.groupby("Week Monday")["Predicted_Date"].min().reset_index()
predicted_values_output["Predicted_Min_Day"] = predicted_values_output["Predicted_Date"].map(lambda x: day_mapper[x.dayofweek])
predicted_values_output

,Week Monday,Predicted_Date,Predicted_Min_Day
0,2015-06-01,2015-06-01,Monday
1,2015-06-08,2015-06-12,Friday
2,2015-06-15,2015-06-19,Friday
3,2015-06-22,2015-06-22,Monday
4,2015-06-29,2015-06-29,Monday
...,...,...,...
475,2024-07-08,2024-07-08,Monday
476,2024-07-15,2024-07-15,Monday
477,2024-07-22,2024-07-22,Monday
478,2024-07-29,2024-08-01,Thursday


In [21]:
# The lowest value day per week: Actual
min_values_per_week = full_calendar.groupby("Week Monday")["Open"].min().reset_index()
actual_values_min_per_week = pd.merge(min_values_per_week, full_calendar, how="left", on=["Week Monday", "Open"])
actual_values_min_per_week = actual_values_min_per_week.groupby("Week Monday")["DayOfWeek"].min().reset_index()
actual_values_min_per_week.rename(columns={"DayOfWeek": "Actual Min Day"}, inplace=True)
actual_values_min_per_week

,Week Monday,Actual Min Day
0,2007-09-17,Tuesday
1,2007-09-24,Monday
2,2007-10-01,Monday
3,2007-10-08,Tuesday
4,2007-10-15,Friday
...,...,...
877,2024-07-08,Monday
878,2024-07-15,Thursday
879,2024-07-22,Thursday
880,2024-07-29,Friday


In [22]:
# Number of values per min day: Actual
actual_values_min_per_week["Actual Min Day"].value_counts()

Actual Min Day
Monday       284
Friday       219
Tuesday      144
Wednesday    123
Thursday     112
Name: count, dtype: int64

In [23]:
# Number of values per min day: Predicted
predicted_values_output["Predicted_Min_Day"].value_counts()

Predicted_Min_Day
Monday       198
Friday       135
Wednesday     55
Tuesday       53
Thursday      39
Name: count, dtype: int64

In [24]:
# Combine the actual and predicted min days per week
combined_prophet_predictions = pd.merge(actual_values_min_per_week, predicted_values_output, how="left", on="Week Monday")
combined_prophet_predictions = combined_prophet_predictions.dropna()
combined_prophet_predictions["Correct Predictions"] = combined_prophet_predictions.apply(lambda x: 1 if x["Actual Min Day"] == x["Predicted_Min_Day"] else 0, axis='columns')
combined_prophet_predictions

,Week Monday,Actual Min Day,Predicted_Date,Predicted_Min_Day,Correct Predictions
402,2015-06-01,Friday,2015-06-01,Monday,0
403,2015-06-08,Friday,2015-06-12,Friday,1
404,2015-06-15,Monday,2015-06-19,Friday,0
405,2015-06-22,Monday,2015-06-22,Monday,1
406,2015-06-29,Monday,2015-06-29,Monday,1
...,...,...,...,...,...
877,2024-07-08,Monday,2024-07-08,Monday,1
878,2024-07-15,Thursday,2024-07-15,Monday,0
879,2024-07-22,Thursday,2024-07-22,Monday,0
880,2024-07-29,Friday,2024-08-01,Thursday,0


In [25]:
# Calculate the percent of accurate calculations
accuracy = combined_prophet_predictions["Correct Predictions"].sum() / len(combined_prophet_predictions)
print(f"Accuracy: {accuracy:.2%}")

Accuracy: 23.75%
